In [1]:
%pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


# Testing ground

In [2]:
import wikipedia

def extract_wikipedia_text_from_search(search_query, num_results):

    search_results = wikipedia.search(search_query, results=num_results)  # Search for the query
    
    text_content_list = []
    for result in search_results:
        page = wikipedia.page(result).content
        text_content_list.append(page)
    
    return text_content_list

In [3]:
search_query = "climate change"  # Replace with your search query
num_results = 100  # Number of search results to retrieve

In [4]:
search_results = wikipedia.search(search_query, results=num_results)
search_results

['Climate change',
 'Climate change denial',
 'Effects of climate change',
 'Climate change mitigation',
 'United Nations Framework Convention on Climate Change',
 'Climate change adaptation',
 'Causes of climate change',
 'Intergovernmental Panel on Climate Change',
 'Climate change feedbacks',
 'Paris Agreement',
 'Climate variability and change',
 'Climate change scenario',
 'Climate change in Africa',
 'Climate change vulnerability',
 '2024 in climate change',
 'Scientific consensus on climate change',
 'Climate change in Bangladesh',
 'Climate change in Pakistan',
 'Politics of climate change',
 'United Nations Climate Change Conference',
 'Climate change and agriculture',
 'Climate change in Greece',
 'Glossary of climate change',
 'Effects of climate change on agriculture',
 'Climate change in Eswatini',
 'Climate change in Botswana',
 'Effects of climate change on human health',
 'Climate change ethics',
 'Climate model',
 'Climate change in South Africa',
 'Climate change art'

In [6]:
len(search_results)

100

In [7]:
text_contents = extract_wikipedia_text_from_search(search_query, num_results)
for text_content in text_contents:
    print(text_content)

c:\Users\fyzan\anaconda3\envs\isl\lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file c:\Users\fyzan\anaconda3\envs\isl\lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


DisambiguationError: "Climate change (disambiguation)" may refer to: 
Climate variability and change
Climate Change (album)
Climate Change: Global Risks, Challenges and Decisions
Climate Change – The Facts
Climate Change TV
Joe Romm
Climate Change (Ladybird Expert book)
Climate Change (children's book)
Global warming (disambiguation)
All pages with titles beginning with Climate change
All pages with titles beginning with Climate Change
All pages with titles containing Climate change

In [ ]:
wikipedia.page('Climate change').summary

# 1. Collecting Data 

In [ ]:
import wikipedia
import wikipediaapi

wiki_wiki = wikipediaapi.Wikipedia('MyProjectName (merlin@example.com)', 'en')

def extract_wikipedia_text_from_search(search_query, num_results):

    search_results = wikipedia.search(search_query, results=num_results)  # Search for the query
    
    text_content_list = []
    for result in search_results:
        page = wiki_wiki.page(result).text
        text_content_list.append(page)
    
    return text_content_list

In [9]:
search_query = "climate change"  # Replace with your search query
num_results = 100  # Number of search results to retrieve
text_contents = extract_wikipedia_text_from_search(search_query, num_results)
for text_content in text_contents:
    print(text_content)

In common usage, climate change describes global warming—the ongoing increase in global average temperature—and its effects on Earth's climate system. Climate change in a broader sense also includes previous long-term changes to Earth's climate. The current rise in global average temperature is primarily caused by humans burning fossil fuels. Fossil fuel use, deforestation, and some agricultural and industrial practices add to greenhouse gases, notably carbon dioxide and methane. Greenhouse gases absorb some of the heat that the Earth radiates after it warms from sunlight. Larger amounts of these gases trap more heat in Earth's lower atmosphere, causing global warming.
Climate change has an increasingly large impact on the environment. Deserts are expanding, while heat waves and wildfires are becoming more common. Amplified warming in the Arctic has contributed to thawing permafrost, retreat of glaciers and sea ice decline. Higher temperatures are also causing more intense storms, drou

# 2. Creating vector embeddings

In [10]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Load the pre-trained model
model = SentenceTransformer('BAAI/bge-base-en-v1.5')

# Convert texts to embeddings
embeddings = model.encode(text_contents)

# # Save embeddings and texts
# np.save("embeddings.npy", embeddings)
# np.save("texts.npy", text_contents)


c:\Users\fyzan\anaconda3\envs\isl\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\fyzan\anaconda3\envs\isl\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\fyzan\anaconda3\envs\isl\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [13]:
# If embeddings is a tensor
# print(embeddings.size())  # This would raise an error
print(embeddings.size)    # This would return the size of the tensor

# Alternatively, you can use .shape to get the dimensions
print(embeddings.shape)

76800
(100, 768)


In [80]:
import faiss
import numpy as np

# # Load embeddings and texts
# embeddings = np.load("embeddings.npy")
# texts = np.load("texts.npy")

# Build the FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

# Save the FAISS index
faiss.write_index(index, "faiss_index.index")

# Querying the index
def search(query, k=3):
    query_embedding = model.encode([query])
    distances, indices = index.search(query_embedding, k)
    
    results = []
    for i in indices[0]:
        results.append(texts[i])
    return results

In [81]:
# Example search
query = "What is cimate change"
results = search(query)
print("Search results:", results)

Search results: ['Climate change education (CCE) is education that aims to address and develop effective responses to climate change. It helps learners understand the causes and consequences of climate change, prepares them to live with the impacts of climate change and empowers learners to take appropriate actions to adopt more sustainable lifestyles. Climate change and climate change education are global challenges that can be anchored in the curriculum in order to provide local learning and widen up mindset shits on how climate change can be mitigated. In such as case CCE is more than climate change literacy but understanding ways of dealing with climate\nCCE helps policymakers understand the urgency and importance of putting mechanisms into place to combat climate change on a national and global scale. Communities learn about how climate change will affect them, what they can do to protect themselves from negative consequences, and how they can reduce their own carbon footprint. In

# gemini pro

In [9]:
import os
from dotenv import load_dotenv

# Import the Python SDK
import google.generativeai as genai

# Load environment variables from .env file
load_dotenv()

# Get API key from environment variable
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

model = genai.GenerativeModel('gemini-pro')

response = model.generate_content("Write a story about a magic backpack.")
print(response.text)

In the quaint town of Evermore, nestled amidst rolling hills and whispering willows, resided a peculiar backpack named Serendipity. Unlike ordinary backpacks, Serendipity possessed a secret power that would forever alter the life of its owner, a young girl named Anya.

Anya was a curious and imaginative child who spent her afternoons exploring the hidden nooks and crannies of her surroundings. One day, as she stumbled upon an abandoned attic, her eyes widened in wonder. There, amidst dusty boxes and forgotten treasures, lay Serendipity, its vibrant hues shimmering in the dim light.

With trembling hands, Anya reached out and grasped the backpack. As her fingertips brushed against its soft leather exterior, she felt a surge of energy coursing through her body. Suddenly, Serendipity began to glow, its interior illuminating with a myriad of colors.

Anya peered into the backpack and gasped in astonishment. Instead of textbooks and pencils, she beheld a miniature world teeming with magical